In [1]:
import pandas as pd
import numpy as np
import boto3
import logging
from io import StringIO
from dotenv import load_dotenv
from pathlib import Path
import os
import polars as pl
from logging import Logger
import json
from chalice import Response
from datetime import datetime, timedelta
from unidecode import unidecode
from functions_mo import (
calculate_volume_distribution_factor,
calculate_volume_distribution_blocks,
get_item_from_dynamodb, 
get_item_from_dynamodb_global, 
get_ssm_parameter,
object_to_dataframe,
group_by_month,
multiply_price,
multiply_by_month_promediado,
multiply_p_social,
total_cost,
calculate_promediados_factor,
group_by_type,
group_by_type_sum,
merge_factor_by_id,
multipy_factor_combined,

) 

In [2]:
#set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
env_path = Path('C:/Users/Nata_/Documents/Etapa_1_proyecto/Simulador_mano_de_obra/mano_de_obra_agrovid/var.env')

load_dotenv(dotenv_path=env_path)
AWS_PROFILE = os.environ["aws_profile"]
SSM_PARAMETER_BUCKET_INPUT_FILES_NAME = os.environ['SSM_PARAMETER_BUCKET_INPUT_FILES_NAME']
SSM_PARAMETER_BUCKET_OUTPUT_FILES_NAME = os.environ['SSM_PARAMETER_BUCKET_OUTPUT_FILES_NAME']
REGION = os.environ['REGION']
SSM_PARAMETER_DYNAMODB_MODULE_CONFIG_NAME = os.environ['SSM_PARAMETER_DYNAMODB_MODULE_CONFIG_NAME']
SSM_PARAMETER_DYNAMODB_GLOBAL_MODULE_CONFIG_NAME = os.environ['SSM_PARAMETER_DYNAMODB_GLOBAL_MODULE_CONFIG_NAME']


### Función para despliegue

In [4]:
def materiales(event, context):
    modulo: str = "modulo_mano_de_obra"
    modulo_revenue: str = "modulo_ingresos"
    PROCESS_TYPE = "er_simulado"
    modulo_global_materiales = "nombre_finca"
    file_revenue: str = "volumen.csv"
    body = event
    # # 4. Body (usually JSON, but can be other formats)
    # if event.get("body"):
    #     body = event["body"]
    #     # Assuming the body is JSON
    #     body = json.loads(body)
    #     print("body_json", body)
    #     print("type body_json", type(body))
    
    session = boto3.Session(profile_name=AWS_PROFILE)

    s3_client = session.client('s3', region_name=REGION)
    ssm_client = session.client("ssm", region_name=REGION)
    dynamodb_client = boto3.resource("dynamodb", region_name=REGION)

    bucket_input_files = get_ssm_parameter(
        parameter_name=SSM_PARAMETER_BUCKET_INPUT_FILES_NAME,
        logger=logger,
        ssm_client=ssm_client,
        with_decryption=True,
    )
    bucket_output_files = get_ssm_parameter(
        parameter_name=SSM_PARAMETER_BUCKET_OUTPUT_FILES_NAME,
        logger=logger,
        ssm_client=ssm_client,
        with_decryption=True,
    )
    dynamodb_table_name_module_config = get_ssm_parameter(
        parameter_name=SSM_PARAMETER_DYNAMODB_MODULE_CONFIG_NAME,
        logger=logger,
        ssm_client=ssm_client,
        with_decryption=True,
    )

    dynamodb_table_name_global_module_config = get_ssm_parameter(
        parameter_name=SSM_PARAMETER_DYNAMODB_GLOBAL_MODULE_CONFIG_NAME,
        logger=logger,
        ssm_client=ssm_client,
        with_decryption=True,
    )
    logger.info(f"{dynamodb_table_name_module_config}")

    module_config = get_item_from_dynamodb(
        table_name=dynamodb_table_name_module_config,
        key={"module": modulo, "process_type": PROCESS_TYPE},
        logger=logger,
        dynamodb_client=dynamodb_client,
    )

    global_config_materiales = get_item_from_dynamodb_global(
        table_name=dynamodb_table_name_global_module_config,
        partition_key_value=modulo_global_materiales,
        logger=logger,
        dynamodb_client=dynamodb_client,
   )
    
    
    input_files_names: dict = module_config["input_file_names_mano_de_obra"]
    farm_order: dict = global_config_materiales["fincas"]

    # Descarga el archivo CSV desde S3
    
    map_with_dfs = {}
    current_date = datetime.now().strftime("%Y-%m-%d")
    partition = body.get("partition", current_date)

    volum_file = object_to_dataframe(
        s3_client=s3_client,
        bucket_name=bucket_input_files,
        folder_name=f"{PROCESS_TYPE}/{modulo_revenue}",
        partition=partition,
        file_name=file_revenue,
    )

    try:
        for key, file in input_files_names.items():
            df = object_to_dataframe(
                s3_client=s3_client,
                bucket_name=bucket_input_files,
                folder_name=f"{PROCESS_TYPE}/{modulo}",
                partition=partition,
                file_name=file,
            )
            logger.info(f"file_key: {file} read successfully")
            map_with_dfs[key] = df
        volum_distribution = map_with_dfs["volum_distribution"]
    except Exception as e:
        logger.error(
            f"Error reding file {file} from {bucket_input_files}: error detail:{e}"
        )
        return Response(
            status_code=500,
            body=json.dumps(
                {
                    "message": f"Error reding file {file} from {bucket_input_files}: error detail:{e}"
                }
            ),
        )

### Importar los archivos

In [5]:
modulo: str = "modulo_mano_de_obra"
PROCESS_TYPE = "er_simulado"
dynamo_global_materiales = "nombre_finca"
modulo_global: str = "archivos_comunes_global"

   
session = boto3.Session(profile_name=AWS_PROFILE)

s3_client = session.client("s3", region_name=REGION)
ssm_client = session.client("ssm", region_name=REGION)
dynamodb_client = session.resource("dynamodb", region_name=REGION)

bucket_input_files = get_ssm_parameter(
        parameter_name=SSM_PARAMETER_BUCKET_INPUT_FILES_NAME,
        logger=logger,
        ssm_client=ssm_client,
        with_decryption=True,
    )
dynamodb_table_name_module_config = get_ssm_parameter(
        parameter_name=SSM_PARAMETER_DYNAMODB_MODULE_CONFIG_NAME,
        logger=logger,
        ssm_client=ssm_client,
        with_decryption=True,
    )

dynamodb_table_name_global_module_config = get_ssm_parameter(
        parameter_name=SSM_PARAMETER_DYNAMODB_GLOBAL_MODULE_CONFIG_NAME,
        ssm_client=ssm_client,
        logger=logger,
        with_decryption=True
    ) 
    
    
logger.info(f"{dynamodb_table_name_module_config}")

module_config = get_item_from_dynamodb(
        table_name=dynamodb_table_name_module_config,
        key={"module": modulo, "process_type": PROCESS_TYPE},
        logger=logger,
        dynamodb_client=dynamodb_client,
    )

module_global_config = get_item_from_dynamodb(
        table_name=dynamodb_table_name_module_config,
        key={"module": modulo_global, "process_type": PROCESS_TYPE},
        logger=logger,
        dynamodb_client=dynamodb_client,)


global_config_materiales = get_item_from_dynamodb_global(
        table_name=dynamodb_table_name_global_module_config,
        partition_key_value=dynamo_global_materiales,
        logger=logger,
        dynamodb_client=dynamodb_client,
   )
 

input_files_names: dict = module_config["input_file_names_mo"]
input_global_file_names: dict = module_global_config["input_file_names_global"]
farm_order: dict = global_config_materiales["fincas"]

    # Descarga el archivo CSV desde S3
map_with_dfs = {}
current_date = datetime.now().strftime("%Y-%m-%d")
partition = "2025-02-19"

#extraer los archivos del modulo global
try:
    for key, file in input_global_file_names.items():
        df = object_to_dataframe(
            s3_client=s3_client,
            bucket_name=bucket_input_files,
            folder_name=f"{PROCESS_TYPE}/{modulo_global}",
            partition=partition,
            file_name=file,
        )

        logger.info(f"file_key: {file} read successfully")
        map_with_dfs[key] = df
    volum_distribution = map_with_dfs["volum_distribution"]
    volum_file = map_with_dfs["volum_file"]
    sku = map_with_dfs["sku"]
    
except Exception as e:
    logger.error(
            f"Error reding file {file} from {bucket_input_files}: error detail:{e}"
    )
     

#extraer los archivos del modulo particular
try:
    for key, file in input_files_names.items():
        df = object_to_dataframe(
            s3_client=s3_client,
            bucket_name=bucket_input_files,
            folder_name=f"{PROCESS_TYPE}/{modulo}",
            partition=partition,
            file_name=file,
        )

        logger.info(f"file_key: {file} read successfully")
        map_with_dfs[key] = df
    cut_emp = map_with_dfs["cut_emp"]
    promediado = map_with_dfs["promediado"]
    revenue_sku = map_with_dfs["revenue_sku"]
    
except Exception as e:
    logger.error(
            f"Error reding file {file} from {bucket_input_files}: error detail:{e}"
    )
     

2025-06-11 19:30:27,495 - INFO - Found credentials in shared credentials file: ~/.aws/credentials


2025-06-11 19:30:28,439 - INFO - Parameter dev-bucket-upload-files-name retrieved successfully.
2025-06-11 19:30:28,552 - INFO - Parameter dev-dynamodb-table-name-module-config retrieved successfully.
2025-06-11 19:30:28,648 - INFO - Parameter dev-dynamodb-table-name-global-config retrieved successfully.
2025-06-11 19:30:28,648 - INFO - dev-module-config
2025-06-11 19:30:29,537 - INFO - file_key: sku.csv read successfully
2025-06-11 19:30:29,834 - INFO - file_key: volumen.csv read successfully
2025-06-11 19:30:29,944 - INFO - file_key: distribucion_volumen.csv read successfully
2025-06-11 19:30:30,069 - INFO - file_key: corte_y_empaque.csv read successfully
2025-06-11 19:30:30,350 - INFO - file_key: input_mo.csv read successfully
2025-06-11 19:30:30,460 - INFO - file_key: curvas_g.csv read successfully


In [6]:
cut_emp.head(1)

,ID,FINCA,GRUPO,LABOR,MAESTRA,TARIFA,REF,PROMEDIADO,PRESTACIONES,FACTOR
0,100001,ENANO,COSECHA,CAJA INTEGRAL CAMPO,REG SUR,820.0,CAJA,G04,0.3887,1.0


In [7]:
volum_distribution.head(1)

,SEMANA,6/01/2024,13/01/2024,20/01/2024,27/01/2024,3/02/2024,10/02/2024,17/02/2024,24/02/2024,2/03/2024,...,26/10/2024,2/11/2024,9/11/2024,16/11/2024,23/11/2024,30/11/2024,7/12/2024,14/12/2024,21/12/2024,28/12/2024
0,ENERO,1,1,1,1,0.6,0,0,0,0.0,...,0,0.0,0,0,0,0,0,0,0,0


In [8]:
volum_file.head(1)

,Unnamed: 0,CONCEPTO,FINCA,6/01/2024,13/01/2024,20/01/2024,27/01/2024,3/02/2024,10/02/2024,17/02/2024,...,26/10/2024,2/11/2024,9/11/2024,16/11/2024,23/11/2024,30/11/2024,7/12/2024,14/12/2024,21/12/2024,28/12/2024
0,0,CAJAS,ENANO,7969.0,8319.0,7813.0,7359.0,7989.0,5479.0,6037.0,...,9164.0,9406.0,10244.0,9851.0,9751.0,9302.0,10343.0,10606.0,10349.0,9936.0


In [9]:
promediado.head(1)

,DETALLE,PROMEDIADO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,LABORES ORDINARIAS,G01,0.224581,0.224581,0.224581,0.224581,0.224581,0.224581,0.224581,0.224581,0.224581,0.224581,0.224581,0.224581


In [10]:
revenue_sku.head(5)

,SKU,Farm,6/01/2024 0:00,13/01/2024 0:00,20/01/2024 0:00,27/01/2024 0:00,3/02/2024 0:00,10/02/2024 0:00,17/02/2024 0:00,24/02/2024 0:00,...,26/10/2024 0:00,2/11/2024 0:00,9/11/2024 0:00,16/11/2024 0:00,23/11/2024 0:00,30/11/2024 0:00,7/12/2024 0:00,14/12/2024 0:00,21/12/2024 0:00,28/12/2024 0:00
0,210801B,ENANO,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,210804S,ENANO,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,210802S,ENANO,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,102200B,ENANO,326.941593,341.30093,320.541431,301.91532,327.762126,224.785166,247.678052,283.822555,...,331.220196,385.896928,420.277284,404.1538,400.051132,381.630154,424.338925,435.128941,424.585085,407.641068
4,102201S,ENANO,326.941593,341.30093,320.541431,301.91532,327.762126,224.785166,247.678052,283.822555,...,331.220196,385.896928,420.277284,404.1538,400.051132,381.630154,424.338925,435.128941,424.585085,407.641068


In [11]:
sku.head(4)

,SKU,RESUMEN CORPORATIVO,TIPO,DESTINO,EMP. ESPECIALES
0,210801B,Fruta premium Reg,PRE,EU,NaN
1,210804S,Fruta premium Reg,PRE,EU,NaN
2,210802S,Fruta Small Reg,ESP,EU,Emp. Especiales
3,102200B,Fruta Comercial Reg,PRE,EU,NaN


## Corte y empaque

### labor 100001

In [12]:
volum_file.columns = list(volum_file.columns[:3]) + [pd.to_datetime(col, dayfirst=True, errors='coerce') for col in volum_file.columns[3:]]
volum_file.head(3)

,Unnamed: 0,CONCEPTO,FINCA,2024-01-06 00:00:00,2024-01-13 00:00:00,2024-01-20 00:00:00,2024-01-27 00:00:00,2024-02-03 00:00:00,2024-02-10 00:00:00,2024-02-17 00:00:00,...,2024-10-26 00:00:00,2024-11-02 00:00:00,2024-11-09 00:00:00,2024-11-16 00:00:00,2024-11-23 00:00:00,2024-11-30 00:00:00,2024-12-07 00:00:00,2024-12-14 00:00:00,2024-12-21 00:00:00,2024-12-28 00:00:00
0,0,CAJAS,ENANO,7969.0,8319.0,7813.0,7359.0,7989.0,5479.0,6037.0,...,9164.0,9406.0,10244.0,9851.0,9751.0,9302.0,10343.0,10606.0,10349.0,9936.0
1,1,CAJAS,EVA,4098.0,6617.0,7238.0,7600.0,6414.0,4585.0,4992.0,...,7407.0,7045.0,7287.0,6578.0,6586.0,6073.0,6218.0,6091.0,5903.0,5791.0
2,2,CAJAS,SAMI,4560.0,8771.0,9549.0,9418.0,8004.0,6011.0,5031.0,...,8233.0,8250.0,8530.0,7911.0,7875.0,7546.0,8216.0,7893.0,7937.0,7645.0


In [13]:
volum_distribution.columns = list(volum_distribution.columns[:1]) + [pd.to_datetime(col, dayfirst=True, errors='coerce') for col in volum_distribution.columns[1:]]
volum_distribution.head(2)

,SEMANA,2024-01-06 00:00:00,2024-01-13 00:00:00,2024-01-20 00:00:00,2024-01-27 00:00:00,2024-02-03 00:00:00,2024-02-10 00:00:00,2024-02-17 00:00:00,2024-02-24 00:00:00,2024-03-02 00:00:00,...,2024-10-26 00:00:00,2024-11-02 00:00:00,2024-11-09 00:00:00,2024-11-16 00:00:00,2024-11-23 00:00:00,2024-11-30 00:00:00,2024-12-07 00:00:00,2024-12-14 00:00:00,2024-12-21 00:00:00,2024-12-28 00:00:00
0,ENERO,1,1,1,1,0.6,0,0,0,0.0,...,0,0.0,0,0,0,0,0,0,0,0
1,FEBRERO,0,0,0,0,0.4,1,1,1,0.8,...,0,0.0,0,0,0,0,0,0,0,0


In [14]:
volum_file_subset = volum_file.iloc[:,3:]
volum_file_subset.head()

,2024-01-06,2024-01-13,2024-01-20,2024-01-27,2024-02-03,2024-02-10,2024-02-17,2024-02-24,2024-03-02,2024-03-09,...,2024-10-26,2024-11-02,2024-11-09,2024-11-16,2024-11-23,2024-11-30,2024-12-07,2024-12-14,2024-12-21,2024-12-28
0,7969.0,8319.0,7813.0,7359.0,7989.0,5479.0,6037.0,6918.0,8895.0,8528.0,...,9164.0,9406.0,10244.0,9851.0,9751.0,9302.0,10343.0,10606.0,10349.0,9936.0
1,4098.0,6617.0,7238.0,7600.0,6414.0,4585.0,4992.0,5411.0,6998.0,5030.0,...,7407.0,7045.0,7287.0,6578.0,6586.0,6073.0,6218.0,6091.0,5903.0,5791.0
2,4560.0,8771.0,9549.0,9418.0,8004.0,6011.0,5031.0,6875.0,8082.0,9268.0,...,8233.0,8250.0,8530.0,7911.0,7875.0,7546.0,8216.0,7893.0,7937.0,7645.0
3,3625.0,3448.0,4037.0,2996.0,3455.0,2949.0,3097.0,3034.0,3528.0,4040.0,...,4869.0,4584.0,4547.0,3897.0,3637.0,3361.0,3460.0,3426.0,3303.0,3147.0
4,5265.0,5243.0,3467.0,3098.0,3646.0,4033.0,4654.0,4113.0,5740.0,5822.0,...,10303.0,9370.0,9802.0,8779.0,8725.0,8484.0,9021.0,8505.0,7837.0,7814.0


In [15]:
group_data = group_by_month(volum_file_subset, volum_file)
month_colums = group_data.iloc[:, 2:].columns

2025-06-11 19:30:30,869 - INFO - Inicia la función agrupando por mes.
2025-06-11 19:30:30,869 - INFO - agrupando las columnas por mes y sumandolas.
c:\Users\Nata_\Documents\Etapa_1_proyecto\Simulador_mano_de_obra\mano_de_obra_agrovid\functions_mo.py:191: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_df = df.groupby(df.columns.to_period('M'), axis=1).sum()
2025-06-11 19:30:30,881 - INFO - Se realizó la agrupación con éxito.


In [16]:
volum_distribution_matrix = volum_distribution.iloc[:, 1:]
volum_file_matrix = volum_file[volum_file['CONCEPTO'] == 'CAJAS'].iloc[:, 3:]
volum_file_subset = volum_file[volum_file['CONCEPTO'] == 'CAJAS']


In [17]:
full_matrix = volum_file_subset.merge(cut_emp, on=['FINCA'], how='inner')
full_matrix_100001 = full_matrix[full_matrix['ID'] == 100001]
factor_100001 = full_matrix_100001.FACTOR.reset_index(drop=True)

In [18]:
quantity_100001 = calculate_volume_distribution_factor(volum_file_matrix, volum_distribution_matrix, volum_file_subset, month_colums, factor_100001)


2025-06-11 19:30:30,953 - INFO - Iniciando el cálculo de la matriz de distribución de volumen.
2025-06-11 19:30:31,094 - INFO - Matriz de resultados generada.
2025-06-11 19:30:31,094 - INFO - Matriz de resultados transpuesta y columnas renombradas.
2025-06-11 19:30:31,094 - INFO - Factores de ajuste aplicados al DataFrame transpuesto.
2025-06-11 19:30:31,094 - INFO - DataFrame final concatenado con los datos de volumen transformados.


In [19]:
labor_100001 = multiply_price(quantity_100001, full_matrix_100001, 'FINCA', 'TARIFA', 'PROMEDIADO', month_colums)

2025-06-11 19:30:31,125 - INFO - Inicia la función para multiplicar los DataFrames.
2025-06-11 19:30:31,125 - INFO - Realizando el merge sobre la columna FINCA
2025-06-11 19:30:31,141 - INFO - Columnas posteriores a la multiplicación: ['FINCA', 'CONCEPTO', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'TARIFA', 'PROMEDIADO']
2025-06-11 19:30:31,141 - INFO - Proceso completado exitosamente.


In [20]:
promediado_100001 = multiply_by_month_promediado(labor_100001, promediado, month_colums)
promediado_100001.head(5)


2025-06-11 19:30:31,164 - INFO - Inicia la función para multiplicar ambos dataframes.
2025-06-11 19:30:31,170 - INFO - Se realiza el merge sobre la columna PROMEDIADO.
2025-06-11 19:30:31,170 - INFO - Columnas después del merge: ['FINCA', 'PROMEDIADO', '2024-01_df1', '2024-02_df1', '2024-03_df1', '2024-04_df1', '2024-05_df1', '2024-06_df1', '2024-07_df1', '2024-08_df1', '2024-09_df1', '2024-10_df1', '2024-11_df1', '2024-12_df1', 'DETALLE', '2024-01_df2', '2024-02_df2', '2024-03_df2', '2024-04_df2', '2024-05_df2', '2024-06_df2', '2024-07_df2', '2024-08_df2', '2024-09_df2', '2024-10_df2', '2024-11_df2', '2024-12_df2']
2025-06-11 19:30:31,183 - INFO - Proceso completado exitosamente


,FINCA,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,7.466254e+06,3.744228e+06,9.668523e+06,4.476646e+06,7.409619e+06,8.963626e+06,9.237351e+06,8.561147e+06,5.824269e+06,6.731973e+06,8.449812e+06,8.231899e+06
1,EVA,6.055110e+06,3.015639e+06,7.579201e+06,3.856923e+06,5.039135e+06,5.438639e+06,6.081306e+06,6.491922e+06,5.202209e+06,5.683533e+06,5.752698e+06,4.791926e+06
2,SAMI,7.640690e+06,3.592951e+06,1.069066e+07,4.367597e+06,6.175218e+06,7.210988e+06,1.017494e+07,9.513937e+06,6.854401e+06,6.747808e+06,6.901673e+06,6.326747e+06
3,VEGA,3.332005e+06,1.730345e+06,5.026883e+06,2.330582e+06,3.214492e+06,3.304601e+06,3.508295e+06,3.760992e+06,3.186631e+06,3.657926e+06,3.369034e+06,2.662381e+06
4,FEDERICA,3.966649e+06,2.455339e+06,6.359870e+06,3.058675e+06,6.786260e+06,7.597614e+06,8.335141e+06,8.296877e+06,6.512210e+06,7.598395e+06,7.756762e+06,6.623410e+06


In [21]:
social_p_100001 = multiply_p_social(labor_100001, promediado_100001,full_matrix_100001, 'FINCA', 'PRESTACIONES', 'LABOR',month_colums)
social_p_100001.head(4)


2025-06-11 19:30:31,216 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:31,228 - INFO - Suma realizada con éxito.
2025-06-11 19:30:31,228 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:31,235 - INFO - Columnas posteriores a la multiplicación: ['FINCA', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'PRESTACIONES', 'LABOR']
2025-06-11 19:30:31,245 - INFO - Proceso completado exitosamente.


,FINCA,LABOR,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,CAJA INTEGRAL CAMPO,1.445732e+07,1.061758e+07,1.495082e+07,1.269451e+07,1.434766e+07,1.551446e+07,1.788680e+07,1.657743e+07,1.373781e+07,1.538119e+07,1.636184e+07,1.634242e+07
1,EVA,CAJA INTEGRAL CAMPO,1.172485e+07,8.551509e+06,1.172002e+07,1.093715e+07,9.757559e+06,9.413326e+06,1.177557e+07,1.257067e+07,1.227054e+07,1.298571e+07,1.113927e+07,9.513194e+06
2,SAMI,CAJA INTEGRAL CAMPO,1.479510e+07,1.018860e+07,1.653139e+07,1.238528e+07,1.195742e+07,1.248095e+07,1.970231e+07,1.842237e+07,1.616760e+07,1.541737e+07,1.336409e+07,1.256021e+07
3,VEGA,CAJA INTEGRAL CAMPO,6.451948e+06,4.906775e+06,7.773267e+06,6.608877e+06,6.224401e+06,5.719682e+06,6.793308e+06,7.282618e+06,7.516365e+06,8.357615e+06,6.523649e+06,5.285504e+06


In [22]:
cost_100001 = total_cost(labor_100001, promediado_100001, social_p_100001, 'FINCA', 'LABOR', month_colums)
cost_100001.head(3)

2025-06-11 19:30:31,260 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:31,276 - INFO - Suma realizada con éxito.
2025-06-11 19:30:31,281 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:31,283 - INFO - Proceso completado exitosamente.


,FINCA,LABOR,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,CAJA INTEGRAL CAMPO,5.165137e+07,3.793320e+07,5.341446e+07,4.535341e+07,5.125956e+07,5.542816e+07,6.390377e+07,5.922581e+07,4.908076e+07,5.495203e+07,5.845560e+07,5.838620e+07
1,EVA,CAJA INTEGRAL CAMPO,4.188910e+07,3.055179e+07,4.187185e+07,3.907493e+07,3.486062e+07,3.363079e+07,4.207033e+07,4.491096e+07,4.383871e+07,4.639377e+07,3.979703e+07,3.398758e+07
2,SAMI,CAJA INTEGRAL CAMPO,5.285811e+07,3.640060e+07,5.906134e+07,4.424863e+07,4.272001e+07,4.459042e+07,7.039001e+07,6.581719e+07,5.776163e+07,5.508129e+07,4.774561e+07,4.487357e+07


### labor 100002 -5

In [23]:

volum_distribution_matrix = volum_distribution.iloc[:, 1:]

volum_file_matrix_10002 = volum_file[volum_file['CONCEPTO'] == 'CORTADOS'].iloc[:, 3:]
volum_file_subset_10002 = volum_file[volum_file['CONCEPTO'] == 'CORTADOS']

full_matrix_100002 = volum_file_subset_10002.merge(cut_emp, on=['FINCA'], how='inner')
full_matrix_100002 = full_matrix[full_matrix['ID'] == 100002]
full_matrix_100003 = full_matrix[full_matrix['ID'] == 100003]
full_matrix_100004 = full_matrix[full_matrix['ID'] == 100004]
full_matrix_100005 = full_matrix[full_matrix['ID'] == 100005]

factor_100002 = full_matrix_100002.FACTOR.reset_index(drop=True)
factor_100003 = full_matrix_100003.FACTOR.reset_index(drop=True)
factor_100004 = full_matrix_100004.FACTOR.reset_index(drop=True)
factor_100005 = full_matrix_100005.FACTOR.reset_index(drop=True)



In [24]:
quantity_100002 = calculate_volume_distribution_factor(volum_file_matrix_10002, volum_distribution_matrix, volum_file_subset_10002, month_colums, factor_100002)
quantity_100002.head(2)

2025-06-11 19:30:31,331 - INFO - Iniciando el cálculo de la matriz de distribución de volumen.
2025-06-11 19:30:31,481 - INFO - Matriz de resultados generada.
2025-06-11 19:30:31,481 - INFO - Matriz de resultados transpuesta y columnas renombradas.
2025-06-11 19:30:31,481 - INFO - Factores de ajuste aplicados al DataFrame transpuesto.
2025-06-11 19:30:31,481 - INFO - DataFrame final concatenado con los datos de volumen transformados.


,FINCA,CONCEPTO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,CORTADOS,41185.4,33787.8,39105.8,38597.6,39453.4,38541.0,44002.8,40826.2,35702.4,41082.0,42300.6,43150.0
1,EVA,CORTADOS,31503.0,26885.8,30498.2,33845.6,27203.4,24359.0,29673.6,31259.4,32634.0,34996.0,29580.0,25384.0


In [25]:
quantity_100003 = calculate_volume_distribution_factor(volum_file_matrix_10002, 
                                                       volum_distribution_matrix, 
                                                       volum_file_subset_10002, month_colums, 
                                                       factor_100003)
quantity_100003.head(2)

2025-06-11 19:30:31,516 - INFO - Iniciando el cálculo de la matriz de distribución de volumen.
2025-06-11 19:30:31,631 - INFO - Matriz de resultados generada.
2025-06-11 19:30:31,631 - INFO - Matriz de resultados transpuesta y columnas renombradas.
2025-06-11 19:30:31,644 - INFO - Factores de ajuste aplicados al DataFrame transpuesto.
2025-06-11 19:30:31,644 - INFO - DataFrame final concatenado con los datos de volumen transformados.


,FINCA,CONCEPTO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,CORTADOS,41185.4,33787.8,39105.8,38597.6,39453.4,38541.0,44002.8,40826.2,35702.4,41082.0,42300.6,43150.0
1,EVA,CORTADOS,31503.0,26885.8,30498.2,33845.6,27203.4,24359.0,29673.6,31259.4,32634.0,34996.0,29580.0,25384.0


In [26]:
quantity_100004 = calculate_volume_distribution_factor(volum_file_matrix_10002, 
                                                       volum_distribution_matrix, 
                                                       volum_file_subset_10002, month_colums, 
                                                       factor_100004)
quantity_100004.head(2)

2025-06-11 19:30:31,667 - INFO - Iniciando el cálculo de la matriz de distribución de volumen.
2025-06-11 19:30:31,793 - INFO - Matriz de resultados generada.
2025-06-11 19:30:31,793 - INFO - Matriz de resultados transpuesta y columnas renombradas.
2025-06-11 19:30:31,793 - INFO - Factores de ajuste aplicados al DataFrame transpuesto.
2025-06-11 19:30:31,798 - INFO - DataFrame final concatenado con los datos de volumen transformados.


,FINCA,CONCEPTO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,CORTADOS,41185.4,33787.8,39105.8,38597.6,39453.4,38541.0,44002.8,40826.2,35702.4,41082.0,42300.6,43150.0
1,EVA,CORTADOS,31503.0,26885.8,30498.2,33845.6,27203.4,24359.0,29673.6,31259.4,32634.0,34996.0,29580.0,25384.0


In [27]:
quantity_100005 = calculate_volume_distribution_factor(volum_file_matrix_10002, 
                                                       volum_distribution_matrix, 
                                                       volum_file_subset_10002, month_colums, 
                                                       factor_100005)
quantity_100005.head(4)

2025-06-11 19:30:31,818 - INFO - Iniciando el cálculo de la matriz de distribución de volumen.
2025-06-11 19:30:31,947 - INFO - Matriz de resultados generada.
2025-06-11 19:30:31,948 - INFO - Matriz de resultados transpuesta y columnas renombradas.
2025-06-11 19:30:31,948 - INFO - Factores de ajuste aplicados al DataFrame transpuesto.
2025-06-11 19:30:31,948 - INFO - DataFrame final concatenado con los datos de volumen transformados.


,FINCA,CONCEPTO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,CORTADOS,20592.7,16893.9,19552.9,19298.8,19726.7,19270.5,22001.4,20413.1,17851.2,20541.0,21150.3,21575.0
1,EVA,CORTADOS,31503.0,26885.8,30498.2,33845.6,27203.4,24359.0,29673.6,31259.4,32634.0,34996.0,29580.0,25384.0
2,SAMI,CORTADOS,42046.0,33821.2,44644.8,39311.6,34298.4,31791.0,47941.8,44405.2,41661.6,41274.2,35514.2,33966.0
3,VEGA,CORTADOS,15553.6,13139.8,16996.6,16655.0,14302.0,12362.0,14297.8,15249.2,16955.6,19318.0,14801.4,12399.0


In [28]:
labor_100002 = multiply_price(quantity_100002, full_matrix_100002, 'FINCA', 'TARIFA', 'PROMEDIADO', month_colums)
labor_100003 = multiply_price(quantity_100003, full_matrix_100003, 'FINCA', 'TARIFA', 'PROMEDIADO', month_colums)
labor_100004 = multiply_price(quantity_100004, full_matrix_100004, 'FINCA', 'TARIFA', 'PROMEDIADO', month_colums)
labor_100005 = multiply_price(quantity_100005, full_matrix_100005, 'FINCA', 'TARIFA', 'PROMEDIADO', month_colums)

2025-06-11 19:30:31,983 - INFO - Inicia la función para multiplicar los DataFrames.
2025-06-11 19:30:31,983 - INFO - Realizando el merge sobre la columna FINCA
2025-06-11 19:30:31,998 - INFO - Columnas posteriores a la multiplicación: ['FINCA', 'CONCEPTO', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'TARIFA', 'PROMEDIADO']
2025-06-11 19:30:32,001 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,001 - INFO - Inicia la función para multiplicar los DataFrames.
2025-06-11 19:30:32,006 - INFO - Realizando el merge sobre la columna FINCA
2025-06-11 19:30:32,014 - INFO - Columnas posteriores a la multiplicación: ['FINCA', 'CONCEPTO', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'TARIFA', 'PROMEDIADO']
2025-06-11 19:30:32,016 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,016 - INFO - Inicia la

In [29]:
promediado_100002 = multiply_by_month_promediado(labor_100002, promediado, month_colums)
promediado_100003 = multiply_by_month_promediado(labor_100003, promediado, month_colums)
promediado_100004 = multiply_by_month_promediado(labor_100004, promediado, month_colums)
promediado_100005 = multiply_by_month_promediado(labor_100005, promediado, month_colums)

2025-06-11 19:30:32,069 - INFO - Inicia la función para multiplicar ambos dataframes.
2025-06-11 19:30:32,076 - INFO - Se realiza el merge sobre la columna PROMEDIADO.
2025-06-11 19:30:32,080 - INFO - Columnas después del merge: ['FINCA', 'PROMEDIADO', '2024-01_df1', '2024-02_df1', '2024-03_df1', '2024-04_df1', '2024-05_df1', '2024-06_df1', '2024-07_df1', '2024-08_df1', '2024-09_df1', '2024-10_df1', '2024-11_df1', '2024-12_df1', 'DETALLE', '2024-01_df2', '2024-02_df2', '2024-03_df2', '2024-04_df2', '2024-05_df2', '2024-06_df2', '2024-07_df2', '2024-08_df2', '2024-09_df2', '2024-10_df2', '2024-11_df2', '2024-12_df2']
2025-06-11 19:30:32,103 - INFO - Proceso completado exitosamente
2025-06-11 19:30:32,130 - INFO - Inicia la función para multiplicar ambos dataframes.
2025-06-11 19:30:32,131 - INFO - Se realiza el merge sobre la columna PROMEDIADO.
2025-06-11 19:30:32,144 - INFO - Columnas después del merge: ['FINCA', 'PROMEDIADO', '2024-01_df1', '2024-02_df1', '2024-03_df1', '2024-04_df1'

In [30]:
social_p_100002 = multiply_p_social(labor_100002, 
                                    promediado_100002,full_matrix_100002, 
                                    'FINCA', 'PRESTACIONES', 
                                    'LABOR',month_colums)

social_p_100003 = multiply_p_social(labor_100003, 
                                    promediado_100003,full_matrix_100003, 
                                    'FINCA', 'PRESTACIONES', 
                                    'LABOR',month_colums)
social_p_100004 = multiply_p_social(labor_100004, 
                                    promediado_100004,full_matrix_100004, 
                                    'FINCA', 'PRESTACIONES', 
                                    'LABOR',month_colums)
social_p_100005 = multiply_p_social(labor_100005, 
                                    promediado_100005,full_matrix_100005, 
                                    'FINCA', 'PRESTACIONES', 
                                    'LABOR',month_colums)

2025-06-11 19:30:32,231 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,249 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,249 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,249 - INFO - Columnas posteriores a la multiplicación: ['FINCA', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'PRESTACIONES', 'LABOR']
2025-06-11 19:30:32,261 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,261 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,265 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,265 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,277 - INFO - Columnas posteriores a la multiplicación: ['FINCA', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024

In [31]:
cost_100002 = total_cost(labor_100002, promediado_100002, social_p_100002, 'FINCA', 'LABOR', month_colums)
cost_100003 = total_cost(labor_100003, promediado_100003, social_p_100003, 'FINCA', 'LABOR', month_colums)
cost_100004 = total_cost(labor_100004, promediado_100004, social_p_100004, 'FINCA', 'LABOR', month_colums)
cost_100005 = total_cost(labor_100005, promediado_100005, social_p_100005, 'FINCA', 'LABOR', month_colums)

2025-06-11 19:30:32,331 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,340 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,340 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,349 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,349 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,349 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,349 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,349 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,364 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,365 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,365 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,365 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,365 - INFO - Inicia la función sumar los dat

### labor promediados

In [32]:
full_matrix_promediados_100006 = full_matrix[full_matrix['ID'] == 100006]
full_matrix_promediados_1000023 = full_matrix[full_matrix['ID'] == 100023]
factor_promediados_100006 = full_matrix_promediados_100006.FACTOR.reset_index(drop=True)
factor_promediados_1000023 = full_matrix_promediados_1000023.FACTOR.reset_index(drop=True)
factor_concat = pd.concat([factor_promediados_100006, factor_promediados_1000023], ignore_index=True)

In [33]:
quantity_promediados = calculate_promediados_factor(volum_distribution_matrix, volum_file_subset, month_colums, factor_concat)

2025-06-11 19:30:32,414 - INFO - Iniciando el cálculo de la matriz de distribución de volumen con 92 factores.
2025-06-11 19:30:32,414 - INFO - Procesando el mes: 0
2025-06-11 19:30:32,431 - INFO - Procesando el mes: 1
2025-06-11 19:30:32,459 - INFO - Procesando el mes: 2
2025-06-11 19:30:32,475 - INFO - Procesando el mes: 3
2025-06-11 19:30:32,491 - INFO - Procesando el mes: 4
2025-06-11 19:30:32,514 - INFO - Procesando el mes: 5
2025-06-11 19:30:32,531 - INFO - Procesando el mes: 6
2025-06-11 19:30:32,548 - INFO - Procesando el mes: 7
2025-06-11 19:30:32,564 - INFO - Procesando el mes: 8
2025-06-11 19:30:32,581 - INFO - Procesando el mes: 9
2025-06-11 19:30:32,598 - INFO - Procesando el mes: 10
2025-06-11 19:30:32,614 - INFO - Procesando el mes: 11
2025-06-11 19:30:32,635 - INFO - Matriz de resultados generada.
2025-06-11 19:30:32,635 - INFO - Matriz de resultados transpuesta y columnas renombradas.
2025-06-11 19:30:32,635 - INFO - DataFrame final concatenado con los datos de volumen

In [34]:
quantity_promediados_100023 = quantity_promediados.iloc[46:, :]
quantity_promediados_100006 = quantity_promediados.iloc[0:46, :]


In [35]:
quantity_promediados_100023.head(4)

,FINCA,CONCEPTO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
46,ENANO,CAJAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,EVA,CAJAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,SAMI,CAJAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,VEGA,CAJAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
labor_promediados_100023 = multiply_price(quantity_promediados_100023, 
                                          full_matrix_promediados_1000023, 
                                          'FINCA', 'TARIFA', 'PROMEDIADO', 
                                          month_colums)

labor_promediados_100023.head(2)

2025-06-11 19:30:32,698 - INFO - Inicia la función para multiplicar los DataFrames.
2025-06-11 19:30:32,698 - INFO - Realizando el merge sobre la columna FINCA
2025-06-11 19:30:32,718 - INFO - Columnas posteriores a la multiplicación: ['FINCA', 'CONCEPTO', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'TARIFA', 'PROMEDIADO']
2025-06-11 19:30:32,721 - INFO - Proceso completado exitosamente.


,FINCA,PROMEDIADO,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,G04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EVA,G04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
labor_promediados_100006 = multiply_price(quantity_promediados_100006, 
                                          full_matrix_promediados_100006, 
                                          'FINCA', 'TARIFA', 'PROMEDIADO', 
                                          month_colums)

2025-06-11 19:30:32,755 - INFO - Inicia la función para multiplicar los DataFrames.
2025-06-11 19:30:32,759 - INFO - Realizando el merge sobre la columna FINCA
2025-06-11 19:30:32,764 - INFO - Columnas posteriores a la multiplicación: ['FINCA', 'CONCEPTO', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'TARIFA', 'PROMEDIADO']
2025-06-11 19:30:32,764 - INFO - Proceso completado exitosamente.


In [38]:
promediado_100023 = multiply_by_month_promediado(labor_promediados_100023, promediado, month_colums)
promediado_100006 = multiply_by_month_promediado(labor_promediados_100006, promediado, month_colums)

2025-06-11 19:30:32,782 - INFO - Inicia la función para multiplicar ambos dataframes.
2025-06-11 19:30:32,782 - INFO - Se realiza el merge sobre la columna PROMEDIADO.
2025-06-11 19:30:32,797 - INFO - Columnas después del merge: ['FINCA', 'PROMEDIADO', '2024-01_df1', '2024-02_df1', '2024-03_df1', '2024-04_df1', '2024-05_df1', '2024-06_df1', '2024-07_df1', '2024-08_df1', '2024-09_df1', '2024-10_df1', '2024-11_df1', '2024-12_df1', 'DETALLE', '2024-01_df2', '2024-02_df2', '2024-03_df2', '2024-04_df2', '2024-05_df2', '2024-06_df2', '2024-07_df2', '2024-08_df2', '2024-09_df2', '2024-10_df2', '2024-11_df2', '2024-12_df2']
2025-06-11 19:30:32,803 - INFO - Proceso completado exitosamente
2025-06-11 19:30:32,803 - INFO - Inicia la función para multiplicar ambos dataframes.
2025-06-11 19:30:32,815 - INFO - Se realiza el merge sobre la columna PROMEDIADO.
2025-06-11 19:30:32,815 - INFO - Columnas después del merge: ['FINCA', 'PROMEDIADO', '2024-01_df1', '2024-02_df1', '2024-03_df1', '2024-04_df1'

In [39]:
social_promediado_100023 = multiply_p_social(labor_promediados_100023, 
                                    promediado_100023 ,full_matrix_promediados_1000023, 
                                    'FINCA', 'PRESTACIONES', 
                                    'LABOR',month_colums)

social_promediado_1000006 = multiply_p_social(labor_promediados_100006,
                                    promediado_100006 ,full_matrix_promediados_100006, 
                                    'FINCA', 'PRESTACIONES', 
                                    'LABOR',month_colums)


2025-06-11 19:30:32,846 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,866 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,866 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,870 - INFO - Columnas posteriores a la multiplicación: ['FINCA', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', 'PRESTACIONES', 'LABOR']
2025-06-11 19:30:32,870 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,870 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,882 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,882 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,882 - INFO - Columnas posteriores a la multiplicación: ['FINCA', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024

In [40]:
cost_promediado_100023 = total_cost(labor_promediados_100023, promediado_100023, social_promediado_100023, 'FINCA', 'LABOR', month_colums)
cost_promediado_100006 = total_cost(labor_promediados_100006, promediado_100006, social_promediado_1000006, 'FINCA', 'LABOR', month_colums)

2025-06-11 19:30:32,904 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,915 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,915 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,915 - INFO - Proceso completado exitosamente.
2025-06-11 19:30:32,915 - INFO - Inicia la función sumar los dataframes de labor y promediados.
2025-06-11 19:30:32,932 - INFO - Suma realizada con éxito.
2025-06-11 19:30:32,932 - INFO - Realizando el merge sobre la columna FINCA y PRESTACIONES
2025-06-11 19:30:32,937 - INFO - Proceso completado exitosamente.


In [41]:
cost_promediado_100006.head(3)

,FINCA,LABOR,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,ENANO,ALIMENTADOR DE COSECHA CAMPO,3.896294e+06,3.295021e+06,3.798078e+06,3.451927e+06,3.896294e+06,3.492234e+06,3.896294e+06,3.726889e+06,3.404381e+06,3.752563e+06,3.557485e+06,3.367251e+06
1,EVA,ALIMENTADOR DE COSECHA CAMPO,2.805331e+06,2.372415e+06,2.734616e+06,2.485387e+06,2.805331e+06,2.514408e+06,2.805331e+06,2.683360e+06,2.451154e+06,2.701845e+06,2.561389e+06,2.424421e+06
2,SAMI,ALIMENTADOR DE COSECHA CAMPO,3.117035e+06,2.636017e+06,3.038462e+06,2.761541e+06,3.117035e+06,2.793787e+06,3.117035e+06,2.981512e+06,2.723505e+06,3.002050e+06,2.845988e+06,2.693801e+06


### cantidades modulo ingreso

In [42]:
revenue_sku_subset = revenue_sku[revenue_sku['SKU']!= 'Exc-']
revenue_sku_subset.loc[:, 'Farm'] = revenue_sku_subset['Farm'].astype(str).apply(unidecode)

In [43]:
input_mo_join = pd.merge(revenue_sku_subset, sku[['SKU', 'TIPO']] ,how = 'left', on = 'SKU')
input_mo_join.rename(columns={'Farm':'FINCA'}, inplace=True)

In [44]:
mo_grouped_total = group_by_type(input_mo_join, farms=farm_order)

2025-06-11 19:30:33,045 - INFO - Iniciando limpieza de columnas TIPO y FINCA...
2025-06-11 19:30:33,048 - INFO - Renombrando columnas de fechas al tipo Timestamp (posición 3 a penúltima)...
2025-06-11 19:30:33,090 - INFO - Obteniendo valores únicos de TIPO...
2025-06-11 19:30:33,090 - INFO - Creando combinaciones TIPO-FINCA completas...
2025-06-11 19:30:33,097 - INFO - Agrupando por TIPO y FINCA...
2025-06-11 19:30:33,105 - INFO - Uniendo con combinaciones completas...
2025-06-11 19:30:33,105 - INFO - Rellenando NaNs con ceros...
2025-06-11 19:30:33,105 - INFO - Proceso de agrupación finalizado exitosamente.


In [45]:
mo_pre_100008 = merge_factor_by_id(100008, mo_grouped_total, cut_emp)
mo_pre_100008_final = multipy_factor_combined(mo_pre_100008, 'PRE')
mo_pre_100008_final_subset = mo_pre_100008_final.iloc[:,2:54]
full_matrix_100008 = full_matrix[full_matrix['ID'] == 100008]
factor_100008 = full_matrix_100008.FACTOR.reset_index(drop=True)

2025-06-11 19:30:33,144 - INFO - Filtrando por TIPO = PRE
2025-06-11 19:30:33,144 - INFO - Aplicando FACTOR a columnas numéricas...
c:\Users\Nata_\Documents\Etapa_1_proyecto\Simulador_mano_de_obra\mano_de_obra_agrovid\venv\lib\site-packages\pandas\core\indexing.py:862: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  diff = Index(key).difference(self.obj.columns, sort=False)


In [46]:
mo_100009 = group_by_type_sum(input_mo_join, farms=farm_order, types_sum=['EDK', 'ESP'])
mo_100009_factor = merge_factor_by_id(100009, mo_100009, cut_emp)
mo_100009_final =  multipy_factor_combined(mo_100009_factor, 'EDK+ESP')
mo_100009_final_subset = mo_100009_final.iloc[:,2:54]

2025-06-11 19:30:33,166 - INFO - Iniciando limpieza de columnas TIPO y FINCA...
2025-06-11 19:30:33,166 - INFO - Obteniendo valores únicos de TIPO...
2025-06-11 19:30:33,166 - INFO - Creando combinaciones TIPO-FINCA completas...
2025-06-11 19:30:33,166 - INFO - Agrupando por TIPO y FINCA...
2025-06-11 19:30:33,175 - INFO - Uniendo con combinaciones completas...
2025-06-11 19:30:33,182 - INFO - Rellenando NaNs con ceros...
2025-06-11 19:30:33,182 - INFO - Sumando tipos especificados: ['EDK', 'ESP']
c:\Users\Nata_\Documents\Etapa_1_proyecto\Simulador_mano_de_obra\mano_de_obra_agrovid\functions_mo.py:428: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  suma_tipos = subset_sum.groupby('FINCA').sum(numeric_only=True).reset_index()
2025-06-11 19:30:33,182 - INFO - Agregando fila de suma de tipos al DataFr

In [47]:
mo_100010 = group_by_type_sum(input_mo_join, farms=farm_order, types_sum=['ESP', 'MG'])
mo_100010_factor = merge_factor_by_id(100010, mo_100010, cut_emp)
mo_100010_final = multipy_factor_combined(mo_100010_factor, 'ESP+MG')
mo_100010_final_subset = mo_100010_final.iloc[:,2:54]

2025-06-11 19:30:33,232 - INFO - Iniciando limpieza de columnas TIPO y FINCA...
2025-06-11 19:30:33,232 - INFO - Obteniendo valores únicos de TIPO...
2025-06-11 19:30:33,232 - INFO - Creando combinaciones TIPO-FINCA completas...
2025-06-11 19:30:33,232 - INFO - Agrupando por TIPO y FINCA...
2025-06-11 19:30:33,248 - INFO - Uniendo con combinaciones completas...
2025-06-11 19:30:33,248 - INFO - Rellenando NaNs con ceros...
2025-06-11 19:30:33,248 - INFO - Sumando tipos especificados: ['ESP', 'MG']
c:\Users\Nata_\Documents\Etapa_1_proyecto\Simulador_mano_de_obra\mano_de_obra_agrovid\functions_mo.py:428: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  suma_tipos = subset_sum.groupby('FINCA').sum(numeric_only=True).reset_index()
2025-06-11 19:30:33,248 - INFO - Agregando fila de suma de tipos al DataFra

In [48]:
mo_100011_factor = merge_factor_by_id(100011, mo_grouped_total, cut_emp)
mo_100011_final = multipy_factor_combined(mo_100011_factor, 'PRE')
mo_100011_final_subset = mo_100011_final.iloc[:,2:54]

2025-06-11 19:30:33,310 - INFO - Filtrando por TIPO = PRE
2025-06-11 19:30:33,310 - INFO - Aplicando FACTOR a columnas numéricas...
c:\Users\Nata_\Documents\Etapa_1_proyecto\Simulador_mano_de_obra\mano_de_obra_agrovid\venv\lib\site-packages\pandas\core\indexing.py:862: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  diff = Index(key).difference(self.obj.columns, sort=False)


In [49]:
mo_100012_factor = merge_factor_by_id(100012, mo_grouped_total, cut_emp)
mo_100012_final = multipy_factor_combined(mo_100012_factor, 'WT')
mo_100012_final_subset = mo_100012_final.iloc[:,2:54]

2025-06-11 19:30:33,336 - INFO - Filtrando por TIPO = WT
2025-06-11 19:30:33,336 - INFO - Aplicando FACTOR a columnas numéricas...
c:\Users\Nata_\Documents\Etapa_1_proyecto\Simulador_mano_de_obra\mano_de_obra_agrovid\venv\lib\site-packages\pandas\core\indexing.py:862: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  diff = Index(key).difference(self.obj.columns, sort=False)


In [50]:
mo_100013_factor = merge_factor_by_id(100013, mo_grouped_total, cut_emp)
mo_100013_final = multipy_factor_combined(mo_100013_factor, 'MG')
mo_100013_final_subset = mo_100013_final.iloc[:,2:54]

2025-06-11 19:30:33,364 - INFO - Filtrando por TIPO = MG
2025-06-11 19:30:33,379 - INFO - Aplicando FACTOR a columnas numéricas...
c:\Users\Nata_\Documents\Etapa_1_proyecto\Simulador_mano_de_obra\mano_de_obra_agrovid\venv\lib\site-packages\pandas\core\indexing.py:862: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  diff = Index(key).difference(self.obj.columns, sort=False)


In [51]:
mo_08_13_concat = pd.concat([mo_pre_100008_final, mo_100009_final, mo_100010_final, 
                                   mo_100011_final, mo_100012_final, mo_100013_final], axis = 0)
mo_08_13_concat_subset = mo_08_13_concat.iloc[:,2:54]

In [52]:
quantity_08_13 = calculate_volume_distribution_blocks(mo_08_13_concat_subset, volum_distribution_matrix, volum_file_subset, month_colums)

2025-06-11 19:30:33,436 - INFO - Iniciando cálculo de volumen por bloques de 46 filas.
2025-06-11 19:30:34,197 - INFO - Cálculo de matriz de volumen completado correctamente.


In [64]:
id_list = [100008,100009, 100010, 100011, 100012, 100013]
full_matrix_block = full_matrix[full_matrix['ID'].isin(id_list)]

In [ ]:
quantity_08_13 #SE AVANZÓ HASTA LA MATRIZ EN BLOQUE CON CANTIDADES AHORA SE ESTÁ ANALIZANDO COMO PROCESAR LA MATRIZ EN BLOQUE PARA LABOR. 

,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,27635.469759,21912.376756,26768.445334,26198.712561,27425.841029,28771.733701,34190.979391,32170.473780,27974.666827,30997.252556,31275.996620,31432.112852
1,21188.005135,16684.399208,19837.658417,21338.908434,17632.913645,16503.517030,21279.671323,23282.481912,24969.890261,25743.482161,20129.808357,17297.669065
2,27607.943880,20526.545417,28893.820231,24952.103196,22312.782452,22595.109701,36764.901556,35748.300791,34070.425965,31007.142306,24937.666853,23582.585349
3,11808.125118,9695.522426,13325.176164,13058.783577,11391.676871,10155.761234,12432.870473,13691.906180,15484.292797,16950.960679,11939.350836,9733.182309
4,11896.356992,11642.995952,14267.154199,14503.962152,20352.636838,19759.937535,24997.877466,27088.255101,30694.967004,33637.081259,23263.262295,20491.855701
...,...,...,...,...,...,...,...,...,...,...,...,...
271,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
272,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
